<a href="https://colab.research.google.com/github/5655530/TFX-pipeline/blob/main/TFX_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **INSTALL**

In [1]:
!pip install tfx
!pip install pyarrow==<10.0.0>
!pip install -U google-cloud-aiplatform "shapely<2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 30.9 MB/s eta 0:00:00
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipython
    Found existing installation: ipython 8.16.1
    Uninstalling ipython-8.16.1:
      Successfully uninstalled ipython-8.16.1
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 4.0.9
    Uninstalling widgetsnbextension-4.0.9:
      Successfully uninstalled widgetsnbextension-4.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 8.1.1
    Uninstalling ipywidgets-

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install pyarrow==<10.0.0>'


In [2]:
!pip install apache-beam[interactive]
!pip install python-snappy

  Using cached ipython-8.16.1-py3-none-any.whl (806 kB)
  Using cached ipywidgets-8.1.1-py3-none-any.whl (139 kB)
  Using cached widgetsnbextension-4.0.9-py3-none-any.whl (2.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 1.1.7
    Uninstalling jupyterlab-widgets-1.1.7:
      Successfully uninstalled jupyterlab-widgets-1.1.7
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.8.1
    Uninstalling ipywidgets-7.8.1:
      Successfully uninstalled ipywidgets-7.8.1
ERROR: pip'

## **1. IMPORT**

In [3]:
# grader-required-cell

import tensorflow as tf
from tfx import v1 as tfx

# TFX libraries
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# For performing feature selection
from sklearn.feature_selection import SelectKBest, f_classif

# For feature visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from google.protobuf.json_format import MessageToDict
from  tfx.proto import example_gen_pb2
from tfx.types import standard_artifacts
from tensorflow_transform.tf_metadata import dataset_metadata, schema_utils
import tensorflow_transform.beam as tft_beam
import os
import pprint
import tempfile
import pandas as pd

# To ignore warnings from TF
tf.get_logger().setLevel('ERROR')

# For formatting print statements
pp = pprint.PrettyPrinter()

# Display versions of TF and TFX related packages
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))
print('TensorFlow Data Validation version: {}'.format(tfdv.__version__))
print('TensorFlow Transform version: {}'.format(tft.__version__))

2023-10-29 02:41:29.490427: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 02:41:31.266572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend
TensorFlow version: 2.13.1
TFX version: 1.14.0
TensorFlow Data Validation version: 1.14.0
TensorFlow Transform version: 1.14.0


<a name='2'></a>
## 2 - Load the dataset

[Cover Type](https://archive.ics.uci.edu/ml/datasets/covertype) 데이터셋의 변형 버전을 사용할 것입니다. 이 데이터셋은 지도학적 변수를 기반으로 한 숲의 지형 유형을 예측하는 모델을 훈련시키는 데 사용될 수 있습니다. 원래 데이터셋에 대한 자세한 내용은 [here](https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.info)여기서 읽을 수 있습니다. 아래는 데이터 컬럼에 대해 개요입니다.

| Column Name | Variable Type | Units / Range | Description |
| --------- | ------------ | ----- | ------------------- |
| Elevation | quantitative |meters | Elevation in meters |
| Aspect | quantitative | azimuth | Aspect in degrees azimuth |
| Slope | quantitative | degrees | Slope in degrees |
| Horizontal_Distance_To_Hydrology | quantitative | meters | Horz Dist to nearest surface water features |
| Vertical_Distance_To_Hydrology | quantitative | meters | Vert Dist to nearest surface water features |
| Horizontal_Distance_To_Roadways | quantitative | meters | Horz Dist to nearest roadway |
| Hillshade_9am | quantitative | 0 to 255 index | Hillshade index at 9am, summer solstice |
| Hillshade_Noon | quantitative | 0 to 255 index | Hillshade index at noon, summer soltice |
| Hillshade_3pm | quantitative | 0 to 255 index | Hillshade index at 3pm, summer solstice |
| Horizontal_Distance_To_Fire_Points | quantitative | meters | Horz Dist to nearest wildfire ignition points |
| Wilderness_Area (4 binary columns) | qualitative | 0 (absence) or 1 (presence) | Wilderness area designation |
| Soil_Type (40 binary columns) | qualitative | 0 (absence) or 1 (presence) | Soil Type designation |
| Cover_Type (7 types) | integer | 1 to 7 | Forest Cover Type designation |

질적 데이터는 이미 원-핫 인코딩 되었습니다(예를 들어, Soil_Type은 1이 특징의 존재를 나타내는 40개의 바이너리 컬럼으로 되어 있습니다). 학습을 위해, 이 데이터셋의 더 원시적인 형식을 보여주는 수정된 버전을 사용할 것입니다. 이것은 여러분에게 다양한 데이터 타입을 처리하는 능력을 연습하게 해줍니다. 만약 원한다면, 이 데이터셋을 준비하기 위한 코드를 [여기서](link not provided) 볼 수 있지만, 이 과제에는 필요하지 않습니다. 주요 변경 사항은 다음과 같습니다:

- Wilderness_Area와 Soil_Type을 문자열로 변환합니다.
- Cover_Type의 범위를 [0, 6]으로 변환합니다.

In [4]:
# # 작업공간 다시 시작하기

# !rm -rf pipeline
# !rm -rf data

In [5]:
# grader-required-cell

# Declare paths to the data
DATA_DIR = './data'
TRAINING_DIR = f'{DATA_DIR}/training'
TRAINING_DATA = f'{TRAINING_DIR}/dataset.csv'

# Create the directory
!mkdir -p {TRAINING_DIR}

In [6]:
# download the dataset
!wget -nc https://storage.googleapis.com/mlep-public/course_2/week3/dataset.csv -P {TRAINING_DIR}

--2023-10-29 02:41:41--  https://storage.googleapis.com/mlep-public/course_2/week3/dataset.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.207, 108.177.12.207, 74.125.26.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32026095 (31M) [text/csv]
Saving to: ‘./data/training/dataset.csv’

dataset.csv         100%[===================>]  30.54M  71.6MB/s    in 0.4s    

2023-10-29 02:41:42 (71.6 MB/s) - ‘./data/training/dataset.csv’ saved [32026095/32026095]



## **3 - 특성 선택**

첫 번째 작업으로, 모델에 공급할 특성의 수를 줄여보겠습니다. 이렇게 하면 모델의 복잡성을 줄이고 훈련 중에 자원을 절약할 수 있습니다. 모든 특성을 사용하여 훈련된 기본 모델이 이미 있다고 가정하고, 특성의 수를 줄이면 더 나은 모델이 생성될지 확인하고 싶다고 합시다. 레이블(이 경우는 Cover_Type)에 대해 큰 예측 가치를 가진 부분 집합을 선택하고 싶을 것입니다. 다음 셀에서 이 작업을 수행해보겠습니다.

In [7]:
# grader-required-cell

# Load the dataset to a dataframe
df = pd.read_csv(TRAINING_DATA)

# Preview the dataset
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,4
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,4
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,1
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,1
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,4


In [8]:
# Show the data type of each column
df.dtypes

Elevation                              int64
Aspect                                 int64
Slope                                  int64
Horizontal_Distance_To_Hydrology       int64
Vertical_Distance_To_Hydrology         int64
Horizontal_Distance_To_Roadways        int64
Hillshade_9am                          int64
Hillshade_Noon                         int64
Hillshade_3pm                          int64
Horizontal_Distance_To_Fire_Points     int64
Wilderness_Area                       object
Soil_Type                             object
Cover_Type                             int64
dtype: object

노트북 시작 부분의 데이터셋 설명과 각 열의 데이터 타입을 살펴보면, 대부분의 특성이 숫자형이고 두 개만이 그렇지 않습니다. 이것은 특성 부분 집합을 선택할 때 고려되어야 합니다. 왜냐하면 숫자형과 범주형 특성은 다르게 점수가 매겨지기 때문입니다. 다음 섹션에서 사용할 수 있도록 숫자형 특성만 포함하는 임시 데이터프레임을 생성해봅시다.

In [9]:
# grader-required-cell

# Copy original dataset
df_num = df.copy()

# Categorical columns
cat_columns = ['Wilderness_Area', 'Soil_Type']

# Label column
label_column = ['Cover_Type']

# Drop the categorical and label columns
df_num.drop(cat_columns, axis=1, inplace=True)
df_num.drop(label_column, axis=1, inplace=True)

# Preview the resuls
df_num.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points
0,2596,51,3,258,0,510,221,232,148,6279
1,2590,56,2,212,-6,390,220,235,151,6225
2,2804,139,9,268,65,3180,234,238,135,6121
3,2785,155,18,242,118,3090,238,238,122,6211
4,2595,45,2,153,-1,391,220,234,150,6172


당신은 데이터셋의 숫자형 속성에 대해 일변량 특성 선택을 수행하기 위해 scikit-learn의 내장 모듈을 사용할 것입니다. 먼저, 입력 특성과 대상 특성을 준비해야 합니다:

In [10]:
# grader-required-cell

# Set the target values
y = df[label_column].values

# Set the input values
X = df_num.values

이후에, **SelectKBest**를 사용하여 대상 변수에 대한 각 입력 특성을 점수 매겨볼 것입니다. 전달할 점수 함수에 주의하고 입력값(숫자형)과 대상값(범주형)에 적합한지 확인하세요.



In [11]:
df_num = df.drop(columns=['Wilderness_Area', 'Soil_Type', 'Cover_Type'])

# 목표 변수를 저장합니다.
target = df['Cover_Type']

# Create SelectKBest object using f_classif (ANOVA statistics) for 8 classes
select_k_best = SelectKBest(score_func=f_classif, k=8)

# Fit and transform the input data using select_k_best
X_new = select_k_best.fit_transform(df_num, target)

# Extract the features which are selected using get_support API
features_mask = select_k_best.get_support()

# Print the results
reqd_cols = pd.DataFrame({'Columns': df_num.columns, 'Retain': features_mask})
print(reqd_cols)

                              Columns  Retain
0                           Elevation    True
1                              Aspect   False
2                               Slope    True
3    Horizontal_Distance_To_Hydrology    True
4      Vertical_Distance_To_Hydrology    True
5     Horizontal_Distance_To_Roadways    True
6                       Hillshade_9am    True
7                      Hillshade_Noon    True
8                       Hillshade_3pm   False
9  Horizontal_Distance_To_Fire_Points    True


In [12]:
# grader-required-cell

# Set the paths to the reduced dataset
TRAINING_DIR_FSELECT = f'{TRAINING_DIR}/fselect'
TRAINING_DATA_FSELECT = f'{TRAINING_DIR_FSELECT}/dataset.csv'

# Create the directory
!mkdir -p {TRAINING_DIR_FSELECT}

In [13]:
# grader-required-cell

# Get the feature names from SelectKBest
feature_names = list(df_num.columns[features_mask])

# Append the categorical and label columns
feature_names = feature_names + cat_columns + label_column

# Select the selected subset of columns
df_select = df[feature_names]

# Write CSV to the created directory
df_select.to_csv(TRAINING_DATA_FSELECT, index=False)

# Preview the results
df_select.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,3,258,0,510,221,232,6279,Rawah,C7745,4
1,2590,2,212,-6,390,220,235,6225,Rawah,C7745,4
2,2804,9,268,65,3180,234,238,6121,Rawah,C4744,1
3,2785,18,242,118,3090,238,238,6211,Rawah,C7746,1
4,2595,2,153,-1,391,220,234,6172,Rawah,C7745,4


## **4 - 데이터 파이프라인**
선택한 특성 부분집합을 준비했으니 이제 데이터 파이프라인을 구축할 차례입니다. 이 단계에서는 데이터를 가져오기, 검증하기, 변환하기를 포함합니다. 비평가 랩에서 이미 접하게 된 TFX 컴포넌트를 사용할 예정이며, 공식 문서에서 해당 컴포넌트들에 대해 참고할 수 있습니다.

### **4.1 - 대화형 컨텍스트 설정**
보통처럼, 먼저 대화형 컨텍스트를 설정하여 노트북에서 파이프라인 컴포넌트를 수동으로 실행하게 됩니다. sqlite 데이터베이스를 작업 공간의 사전 정의된 디렉토리에 저장할 것입니다. 이 경로는 나중에 ML 메타데이터와 관련된 연습에서 필요하기 때문에 수정하지 않아야 합니다."

In [14]:
# grader-required-cell

# Location of the pipeline metadata store
PIPELINE_DIR = './pipeline'

# Declare the InteractiveContext and use a local sqlite file as the metadata store.
context = InteractiveContext(pipeline_root=PIPELINE_DIR)

## **4.2 - Examples 생성**
파이프라인의 첫 번째 단계는 데이터를 가져오는 것입니다. ExampleGen을 사용하면 원시 데이터를 TFRecords로 변환하여 파이프라인의 나중 단계에서 더 빠른 계산을 할 수 있습니다.

#### **2: ExampleGen**
앞서 불러온 데이터셋을 ExampleGen을 사용하여 가져와보세요. 주의할 점은:

- 입력 데이터는 CSV 형식이므로 이를 처리하기 위해 적절한 유형의 ExampleGen을 사용해야 합니다.
- 이 함수는 훈련 데이터의 디렉토리 경로를 받아들이며, CSV 파일 경로 자체를 받아들이지 않습니다.

이 작업을 실행하는 데는 몇 분 정도 걸릴 수 있습니다."


In [15]:
# 데이터셋에 서로 다른 헤더가 있다는 오류 메시지가 표시되면 이 코드를 주석 해제하고 실행하세요.
# 이는 주로 해당 폴더에 저장된 노트북 체크포인트 때문입니다.
!rm -rf {TRAINING_DIR}/.ipynb_checkpoints
!rm -rf {TRAINING_DIR_FSELECT}/.ipynb_checkpoints
!rm -rf {SERVING_DIR}/.ipynb_checkpoints

In [16]:
!ls ./data/training/

dataset.csv  fselect


In [17]:
!ls ./data/training/fselect
!mv ./data/training/fselect ./data/fselect_backup

dataset.csv


In [18]:
from tfx.components import CsvExampleGen

# CsvExampleGen 인스턴스 생성
example_gen = CsvExampleGen(input_base=TRAINING_DIR)

# 컴포넌트 실행
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## **4.3 - 통계 계산**

다음으로, 데이터의 통계를 계산할 것입니다. 이를 통해 통합된 FACETS 라이브러리 제공의 시각화를 통해 데이터의 특성을 관찰하고 분석할 수 있습니다.

In [19]:
from tfx.components import StatisticsGen

# ExampleGen에서 가져온 데이터셋과 함께 StatisticsGen 인스턴스를 생성합니다.
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

# 컴포넌트를 실행합니다.
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [20]:
# Display the results
context.show(statistics_gen.outputs['statistics'])

표시를 로드하면 Cover_type의 0 열이 빨간색으로 강조 표시될 수 있습니다. 시각화는 이것이 잠재적인 문제가 될 수 있다는 것을 알려줍니다. 그러나 우리의 경우 Cover_Type이 [0, 6]의 범위를 가지므로 이 열에 0이 있을 것으로 예상합니다."

이 해석은 데이터의 시각화에서 'Cover_type' 열의 0 값이 문제가 될 수 있다는 경고나 강조 표시에 대해 설명하고 있습니다. 그러나 실제 데이터셋의 특성을 고려하면 이는 예상되는 것이므로 특별한 문제로 간주하지 않아도 됩니다.

## **4.4 - 스키마 추론**


훈련 및 서비스 중에 들어오는 데이터셋을 검증하기 위해 스키마를 생성해야 합니다. 다행히도 TFX는 SchemaGen 컴포넌트를 사용하여 이 스키마의 첫 번째 초안을 추론할 수 있게 해줍니다.

In [21]:
from tfx.components import SchemaGen

# StatisticsGen의 출력 통계를 사용하여 SchemaGen 인스턴스 생성
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])

# 컴포넌트 실행
context.run(schema_gen)


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [22]:
# Visualize the output
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,,-
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_3pm',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## **4.5 스키마 조정**
스키마의 자동 추론은 데이터 타입을 올바르게 포착할 수 있으며, 질적(즉, 문자열) 데이터에 대한 예상 값을 표시할 수 있습니다. 그러나 이를 더 세밀하게 조정할 수 있습니다. 예를 들어, 우리는 특정 범위의 값을 기대하는 특성들이 있습니다:

- Hillshade_9am: 0에서 255까지
- Hillshade_Noon: 0에서 255까지
- Slope: 0에서 90까지
- Cover_Type: 0에서 6까지

이러한 정보를 기반으로 스키마를 업데이트하려고 합니다. 그렇게 하면 파이프라인이 모델에 유효하지 않은 값이 공급되는 경우 이를 감지할 수 있습니다.

- **tfdv.set_domain()**을 사용하여 특정 특성에 대한 허용되는 값들을 정의할 수 있습니다.
- 변경 후에도 이 값들은 INT 타입이어야 합니다.
- **Cover_Type**-을 범주형 변수로 선언하세요. 다른 네 가지 특성과 달리 여기서의 정수 0부터 6은 양적인 측정이 아닌 지정된 레이블에 해당합니다. set_domain()에 대한 사용 가능한 플래그를 공식 문서에서 확인하여 어떻게 설정하는지 알 수 있습니다.

여기서, TFDV는 TensorFlow Data Validation의 약자로, 데이터의 유효성을 검사하고 데이터 품질을 향상시키는 도구입니다. tfdv.set_domain()은 TFDV에서 제공하는 함수로, 특정 특성에 대해 허용되는 값의 범위나 집합을 지정하는 데 사용됩니다.

In [23]:
# grader-required-cell

try:
    # 스키마 가져오기
    schema_uri = schema_gen.outputs['schema']._artifacts[0].uri

# context.run() 이 노트북 외부에서 작동하지 않는 경우
except IndexError:
    print("context.run() was no-op")
    schema_path = './pipeline/SchemaGen/schema'
    dir_id = os.listdir(schema_path)[0]
    schema_uri = f'{schema_path}/{dir_id}'

In [24]:
# grader-required-cell

# Get the schema pbtxt file from the SchemaGen output
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))

In [25]:


# Set the two `Hillshade` features to have a range of 0 to 255
tfdv.set_domain(schema, 'Hillshade_9am', schema_pb2.IntDomain(name='Hillshade_9am', min=0, max=255))
tfdv.set_domain(schema, 'Hillshade_Noon', schema_pb2.IntDomain(name='Hillshade_Noon', min=0, max=255))

# Set the `Slope` feature to have a range of 0 to 90
tfdv.set_domain(schema, 'Slope', schema_pb2.IntDomain(name='Slope', min=0, max=90))

# Set `Cover_Type` to categorical having minimum value of 0 and maximum value of 6
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))


tfdv.display_schema(schema=schema)


,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,,-
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_3pm',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## **4.6 - 스키마 환경**

지도 학습에서는 해당 레이블과 함께 일련의 특징을 제공함으로써 모델에 예측을 학습시킵니다. 따라서 우리의 훈련 데이터셋에는 입력 특징과 레이블이 모두 있을 것이고, 스키마는 이것들을 감지하기 위해 구성됩니다.

그러나 훈련 후 모델을 추론을 위해 제공할 때, 들어오는 데이터에는 더 이상 레이블이 없게 됩니다. 이것은 현재 버전의 스키마를 사용하여 데이터를 검증할 때 문제를 일으킬 것입니다. 다음 셀에서 이를 시연해 보겠습니다. 레이블 열 (즉, Cover_Type)을 제거하여 훈련 세트의 부분 집합을 얻어 서빙 데이터셋을 시뮬레이션합니다. 그 후에는 이전에 정제한 스키마를 사용하여 이 서빙 데이터셋을 검증하게 됩니다.

In [26]:
# grader-required-cell

# Declare paths to the serving data
SERVING_DIR = f'{DATA_DIR}/serving'
SERVING_DATA = f'{SERVING_DIR}/serving_dataset.csv'

# Create the directory
!mkdir -p {SERVING_DIR}

In [27]:
# grader-required-cell

# Read a subset of the training dataset
serving_data = pd.read_csv(TRAINING_DATA, nrows=100)

# Drop the `Cover_Type` column
serving_data.drop(columns='Cover_Type', inplace=True)

# Save the modified dataset
serving_data.to_csv(SERVING_DATA, index=False)

# Delete unneeded variable from memory
del serving_data

In [28]:
# grader-required-cell

# Declare StatsOptions to use the curated schema
stats_options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)

# Compute the statistics of the serving dataset
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA, stats_options=stats_options)

# Detect anomalies in the serving dataset
anomalies = tfdv.validate_statistics(serving_stats, schema=schema)

# Display the anomalies detected
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


예상대로, 누락된 열이 플래그로 표시됩니다. 이 문제를 해결하기 위해서는 스키마가 훈련 중인지 아니면 추론/서빙을 위해 사용되는 중인지 감지하도록 구성해야 합니다. 이것은 스키마 환경을 설정하여 수행할 수 있습니다.

- SERVING 환경에서 유효성 검사할 때 Cover_Type 피처를 무시하도록 합니다.

In [29]:
# grader-required-cell

schema.default_environment.append('TRAINING')


# Create another default schema environment with name SERVING
schema.default_environment.append('SERVING')

# Remove Cover_Type feature from SERVING using TFDV
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')



In [30]:
# grader-required-cell

# Validate the serving dataset statistics in the `SERVING` environment
anomalies = tfdv.validate_statistics(serving_stats, schema=schema, environment='SERVING')

# Display the anomalies detected
tfdv.display_anomalies(anomalies)

이제 우리는 이 정제된 스키마를 로컬 디렉토리에 저장할 수 있으므로, 우리의 TFX 파이프라인에 임포트할 수 있습니다.

In [31]:
# grader-required-cell

# Declare the path to the updated schema directory
UPDATED_SCHEMA_DIR = f'{PIPELINE_DIR}/updated_schema'

# Create the said directory
!mkdir -p {UPDATED_SCHEMA_DIR}

# Declare the path to the schema file
schema_file = os.path.join(UPDATED_SCHEMA_DIR, 'schema.pbtxt')

# Save the curated schema to the said file
tfdv.write_schema_text(schema, schema_file)

정상 작동하는지 확인하기 위해 방금 저장한 스키마를 표시하고 우리가 도입한 변경 사항이 포함되어 있는지 확인해봅시다. 도메인 열에 범위가 여전히 표시되어야 하며 사용 가능한 환경이 두 개 있어야 합니다.

In [32]:
# grader-required-cell

# Load the schema from the directory we just created
new_schema = tfdv.load_schema_text(schema_file)

# Display the schema. Check that the Domain column still contains the ranges.
tfdv.display_schema(schema=new_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,,-
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_3pm',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [33]:
# The environment list should show `TRAINING` and `SERVING`.
new_schema.default_environment

['TRAINING', 'SERVING']

## **4.7 - 업데이트된 스키마를 사용하여 새로운 통계 생성하기**

이제 방금 정리한 스키마를 사용하여 통계를 계산할 것입니다. 그러나 TFX 컴포넌트들은 메타데이터 스토어에서 아티팩트 정보를 얻어 서로 상호 작용한다는 것을 기억하세요. 그래서 우선 정리된 스키마 파일을 ML 메타데이터에 가져와야 합니다. 이를 위해 ImportSchemaGen을 사용하여 정리된 스키마를 나타내는 아티팩트를 생성할 것입니다.

In [34]:
from tfx.v1.components import ImportSchemaGen

# Use ImportSchemaGen to import the curated schema into ML Metadata
user_schema_importer = ImportSchemaGen(schema_file=schema_file)

# Run the component
context.run(user_schema_importer, enable_cache=False)

# Show the imported schema
context.show(user_schema_importer.outputs['schema'])


,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,,-
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_3pm',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


아티팩트를 성공적으로 생성한 후, 이제 StatisticsGen을 사용하고 스키마 매개변수를 전달하여 정리된 스키마를 사용할 수 있습니다.

이제 새로운 스키마를 사용한 통계
이전 섹션에서 업데이트한 스키마를 사용하여 StatisticsGen으로 통계를 계산해보겠습니다.

In [35]:
from tfx.components import StatisticsGen

# grader-required-cell

### START CODE HERE ###
# Use StatisticsGen to compute the statistics using the curated schema
statistics_gen_updated = StatisticsGen(
    examples=example_gen.outputs['examples'],  # Assuming example_gen is your ExampleGen component.
    schema=user_schema_importer.outputs['schema']
)

# Run the component
context.run(statistics_gen_updated)
### END CODE HERE ###


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 5
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [36]:
context.show(statistics_gen_updated.outputs['statistics'])


차트는 이전 실행과 대부분 동일하게 보일 것입니다. 하지만 'Cover Type'이 범주형 특징 아래에 있다는 것을 확인할 수 있습니다. 이것은 StatisticsGen이 업데이트된 스키마를 실제로 사용하고 있음을 보여줍니다.


## **4.8 - Check anomalies**

이제 데이터셋이 스키마와 관련하여 어떠한 이상치가 있는지 확인하겠습니다. ExampleValidator 컴포넌트를 사용하면 이것을 쉽게 할 수 있습니다.

#### **ExampleValidator**
ExampleValidator를 사용하여 이상치가 있는지 확인합니다. 이전 섹션에서의 업데이트된 통계와 스키마를 전달해야 합니다.



In [37]:
from tfx.components import ExampleValidator

# grader-required-cell


# Use ExampleValidator to check for anomalies in the data.
# We use the statistics from the updated StatisticsGen and the curated schema.
example_validator = ExampleValidator(
    statistics=statistics_gen_updated.outputs['statistics'],
    schema=user_schema_importer.outputs['schema']
)

# Run the component.
context.run(example_validator)



ExecutionResult(
    component_id: ExampleValidator
    execution_id: 6
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [38]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

## **4.10 - 특성 엔지니어링**


이제 모델을 훈련시키기에 적합한 형태로 특성들을 변환하는 작업을 시작할 것입니다. 이 작업은 스케일링 및 문자열을 어휘 인덱스로 변환하는 것과 같은 여러 방법을 포함할 수 있습니다. 이러한 변환들은 훈련 데이터에서 물론, 모델이 추론을 위해 배포될 때 서빙 데이터에서도 일관성 있게 이루어져야 합니다. TFX는 훈련과 추론 중에 들어오는 데이터를 처리할 그래프를 생성함으로써 이를 보장합니다.

먼저 이 연습을 위해 사용할 상수와 유틸리티 함수를 선언해봅시다.

In [39]:
# grader-required-cell

# Set the constants module filename
_cover_constants_module_file = 'cover_constants.py'

In [40]:
%%writefile {_cover_constants_module_file}

SCALE_MINMAX_FEATURE_KEYS = [
        "Horizontal_Distance_To_Hydrology",
        "Vertical_Distance_To_Hydrology",
    ]

SCALE_01_FEATURE_KEYS = [
        "Hillshade_9am",
        "Hillshade_Noon",
        "Horizontal_Distance_To_Fire_Points",
    ]

SCALE_Z_FEATURE_KEYS = [
        "Elevation",
        "Slope",
        "Horizontal_Distance_To_Roadways",
    ]

VOCAB_FEATURE_KEYS = ["Wilderness_Area"]

HASH_STRING_FEATURE_KEYS = ["Soil_Type"]

LABEL_KEY = "Cover_Type"

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Writing cover_constants.py


<a name='ex-10'></a>
#### Exercise 10: 전처리 함수

특성을 변환하기 위해 모듈을 완성하세요. 코드의 주석을 참조하여 어떤 작업을 수행해야 하는지 힌트를 얻을 수 있습니다.

다음은 이 함수를 완성하는 데 필요한 함수들에 대한 문서 링크입니다:

- [`tft.scale_by_min_max`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_by_min_max)
- [`tft.scale_to_0_1`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_0_1)
- [`tft.scale_to_z_score`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_z_score)
- [`tft.compute_and_apply_vocabulary`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/compute_and_apply_vocabulary)
- [`tft.hash_strings`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/hash_strings)

In [41]:
# grader-required-cell

# Set the transform module filename
_cover_transform_module_file = 'cover_transform.py'

In [42]:
%%writefile {_cover_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import cover_constants

_SCALE_MINMAX_FEATURE_KEYS = cover_constants.SCALE_MINMAX_FEATURE_KEYS
_SCALE_01_FEATURE_KEYS = cover_constants.SCALE_01_FEATURE_KEYS
_SCALE_Z_FEATURE_KEYS = cover_constants.SCALE_Z_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = cover_constants.VOCAB_FEATURE_KEYS
_HASH_STRING_FEATURE_KEYS = cover_constants.HASH_STRING_FEATURE_KEYS
_LABEL_KEY = cover_constants.LABEL_KEY
_transformed_name = cover_constants.transformed_name

def preprocessing_fn(inputs):

    features_dict = {}

    ### START CODE HERE ###
    for feature in _SCALE_MINMAX_FEATURE_KEYS:
        data_col = inputs[feature]
        # Transform using scaling of min_max function
        features_dict[_transformed_name(feature)] = tft.scale_by_min_max(data_col)

    for feature in _SCALE_01_FEATURE_KEYS:
        data_col = inputs[feature]
        # Transform using scaling of 0 to 1 function
        features_dict[_transformed_name(feature)] = tft.scale_to_0_1(data_col)

    for feature in _SCALE_Z_FEATURE_KEYS:
        data_col = inputs[feature]
        # Transform using scaling to z score
        features_dict[_transformed_name(feature)] = tft.scale_to_z_score(data_col)

    for feature in _VOCAB_FEATURE_KEYS:
        data_col = inputs[feature]
        # Transform using vocabulary available in column
        features_dict[_transformed_name(feature)] = tft.compute_and_apply_vocabulary(data_col)

    for feature in _HASH_STRING_FEATURE_KEYS:
        data_col = inputs[feature]
        # Transform by hashing strings into buckets
        features_dict[_transformed_name(feature)] = tft.hash_strings(data_col, hash_buckets=10)
    ### END CODE HERE ###

    # No change in the label
    features_dict[_LABEL_KEY] = inputs[_LABEL_KEY]

    return features_dict



Writing cover_transform.py


In [46]:
# Test your preprocessing_fn

import cover_transform
from testing_values import feature_description, raw_data

# NOTE: These next two lines are for reloading your cover_transform module in case you need to
# update your initial solution and re-run this cell. Please do not remove them especially if you
# have revised your solution. Else, your changes will not be detected.
import importlib
importlib.reload(cover_transform)

raw_data_metadata = dataset_metadata.DatasetMetadata(schema_utils.schema_from_feature_spec(feature_description))

with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
    transformed_dataset, _ = (
        (raw_data, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(cover_transform.preprocessing_fn))

transformed_data, transformed_metadata = transformed_dataset

2023-10-29 03:02:34.163198: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-29 03:02:34.764168: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-29 03:02:34.764454: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [47]:
# Test that the transformed data matches the expected output
transformed_data

[{'Cover_Type': 4,
  'Elevation_xf': 0.0,
  'Hillshade_9am_xf': 1.0,
  'Hillshade_Noon_xf': 1.0,
  'Horizontal_Distance_To_Fire_Points_xf': 1.0,
  'Horizontal_Distance_To_Hydrology_xf': 1.0,
  'Horizontal_Distance_To_Roadways_xf': 0.0,
  'Slope_xf': 0.0,
  'Soil_Type_xf': 4,
  'Vertical_Distance_To_Hydrology_xf': 0.5,
  'Wilderness_Area_xf': 0}]

**Expected Output:**

```
[{'Cover_Type': 4,
  'Elevation_xf': 0.0,
  'Hillshade_9am_xf': 1.0,
  'Hillshade_Noon_xf': 1.0,
  'Horizontal_Distance_To_Fire_Points_xf': 1.0,
  'Horizontal_Distance_To_Hydrology_xf': 1.0,
  'Horizontal_Distance_To_Roadways_xf': 0.0,
  'Slope_xf': 0.0,
  'Soil_Type_xf': 4,
  'Vertical_Distance_To_Hydrology_xf': 0.5,
  'Wilderness_Area_xf': 0}]
```

In [48]:
# Test that the transformed metadata's schema matches the expected output
MessageToDict(transformed_metadata.schema)

{'feature': [{'name': 'Cover_Type',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Elevation_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Hillshade_9am_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Hillshade_Noon_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Fire_Points_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Hydrology_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Roadways_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Slope_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Soil_Type_xf',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 

**Expected Output:**

```
{'feature': [{'name': 'Cover_Type',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Elevation_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Hillshade_9am_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Hillshade_Noon_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Fire_Points_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Hydrology_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Roadways_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Slope_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Soil_Type_xf',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Vertical_Distance_To_Hydrology_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Wilderness_Area_xf',
   'type': 'INT',
   'intDomain': {'isCategorical': True},
   'presence': {'minFraction': 1.0},
   'shape': {}}]}
```

<a name='ex-11'></a>
#### Exercise 11: Transform

[TFX Transform component](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/Transform)를 사용하여 변환을 수행하고 변환 그래프를 생성하세요. 이를 위해 데이터셋 예제, 정제된 스키마, 그리고 전처리 함수가 포함된 모듈을 전달해야 합니다.

In [49]:
# grader-required-cell

from tfx.components import Transform

### START CODE HERE ###
# Instantiate the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=user_schema_importer.outputs['schema'],
    module_file=_cover_transform_module_file
)
### END CODE HERE ###

# Run the component
context.run(transform, enable_cache=False)


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying cover_transform.py -> build/lib
copying cover_constants.py -> build/lib
copying testing_values.py -> build/lib
installing to /tmp/tmpv_w31auu
running install
running install_lib
copying build/lib/cover_transform.py -> /tmp/tmpv_w31auu
copying build/lib/cover_constants.py -> /tmp/tmpv_w31auu
copying build/lib/testing_values.py -> /tmp/tmpv_w31auu
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info


/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpv_w31auu/tfx_user_code_Transform-0.0+101b8af51127da0a900ad281ea6baaf9a7fbb37d9a3f9fbf3cffcc36db1a2703-py3.10.egg-info
running install_scripts
creating /tmp/tmpv_w31auu/tfx_user_code_Transform-0.0+101b8af51127da0a900ad281ea6baaf9a7fbb37d9a3f9fbf3cffcc36db1a2703.dist-info/WHEEL
creating '/tmp/tmp99hjctnt/tfx_user_code_Transform-0.0+101b8af51127da0a900ad281ea6baaf9a7fbb37d9a3f9fbf3cffcc36db1a2703-py3-none-any.whl' and adding '/tmp/tmpv_w31auu' to it
adding 'cover_constants.py'
adding 'cover_transform.py'
adding 'testing_values.py'
a

ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

변환된 데이터셋의 몇 가지 예제를 검사하여 변환이 올바르게 수행되었는지 확인해봅시다.

In [50]:
# grader-required-cell

try:
    transform_uri = transform.outputs['transformed_examples'].get()[0].uri

# for grading since context.run() does not work outside the notebook
except IndexError:
    print("context.run() was no-op")
    examples_path = './pipeline/Transform/transformed_examples'
    dir_id = os.listdir(examples_path)[0]
    transform_uri = f'{examples_path}/{dir_id}'

In [51]:
# grader-required-cell

# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform_uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [54]:
# grader-required-cell

# import helper function to get examples from the dataset
from util import get_records

# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'Cover_Type': {'int64List': {'value': ['1']}},
                           'Elevation_xf': {'floatList': {'value': [-0.55514824]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.9212598]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.93700784]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [0.8533389]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [0.19378163]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [0.53379315]}},
                           'Slope_xf': {'floatList': {'value': [-0.6813106]}},
                           'Soil_Type_xf': {'int64List': {'value': ['4']}},
                           'Vertical_Distance_To_Hydrology_xf': {'floatList': {'value': [0.3019608]}},
                           'Wilderness_Area_xf': {'int64List': {'value': ['0']}}}}},
 {'feat

<a name='5'></a>
## 5 - ML Metadata

TFX는 각 구성 요소가 사용하는 아티팩트의 기록을 유지하기 위해 내부적으로 [ML Metadata](https://www.tensorflow.org/tfx/guide/mlmd)를 사용합니다. 이를 통해 파이프라인이 어떻게 실행되는지 추적하기가 더 쉬워져 필요한 경우 문제를 해결하거나 결과를 재현하고자 할 때 도움이 됩니다.

이 과제의 마지막 섹션에서는 이 메타데이터 스토어를 통해 관련 아티팩트를 검색하는 방법을 보여줍니다. 이 기술은 파이프라인의 특정 단계에 어떤 입력이 공급되었는지 알고 싶을 때 유용합니다. 예를 들면, 특성 변환을 수행하기 위해 사용된 스키마의 위치를 알 수 있고, 어떤 예제 집합이 모델을 훈련하는 데 사용되었는지 확인할 수 있습니다.l.

먼저 관련 모듈을 가져오고 메타데이터 스토어에 연결을 설정할 것입니다. 또한 아티팩트 정보를 표시하는 데 도움이 되는 몇몇 도우미 함수를 제공했으며, 이는 실습 작업 공간의 외부 util.py 모듈에서 코드를 검토할 수 있습니다.

In [ ]:
# grader-required-cell

# Import mlmd and utilities
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2
from util import display_types, display_artifacts, display_properties

# Get the connection config to connect to the metadata store
connection_config = context.metadata_connection_config

# Instantiate a MetadataStore instance with the connection config
store = mlmd.MetadataStore(connection_config)

# Declare the base directory where All TFX artifacts are stored
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

<a name='5-1'></a>
#### 5.1 - 저장된 아티팩트에 액세스하기

연결이 설정되면 이제 메타데이터 스토어와 상호 작용할 수 있습니다. 예를 들어, get_artifact_types() 함수를 사용하여 저장된 모든 아티팩트 유형을 검색할 수 있습니다. 참고로 API는 [여기](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd/MetadataStore)에 문서화되어 있습니다.

In [ ]:
# grader-required-cell

# Get the artifact types
types = store.get_artifact_types()

# Display the results
display_types(types)

특정 유형에 대한 아티팩트 목록도 가져올 수 있어 파이프라인에서 사용된 변형이 있는지 확인할 수 있습니다. 예를 들어, 이전 과제의 일부에서 스키마를 정제했으므로 이것은 기록에 나타나야 합니다. 아래의 셀을 실행하면 최소 두 개의 행이 표시되어야 합니다: 하나는 유추된 스키마용이고 다른 하나는 업데이트된 스키마용입니다. 이 노트북을 이전에 실행했다면 ./SchemaGen/schema 디렉토리 아래에 저장된 다양한 스키마 아티팩트 때문에 더 많은 행을 볼 수 있습니다.

In [ ]:
# grader-required-cell

# Retrieve the transform graph list
schema_list = store.get_artifacts_by_type('Schema')

# Display artifact properties from the results
display_artifacts(store, schema_list, base_dir)

게다가, 특정 아티팩트의 속성도 얻을 수 있습니다. TFX는 각 컴포넌트에 대해 자동으로 몇몇 속성을 선언합니다. 대부분의 아티팩트 유형에 대해 `name`, `state`, `producer_component`를 볼 수 있을 것입니다. 적절한 곳에 추가 속성이 추가됩니다. 예를 들어, `ExampleStatistics` 아티팩트에는 통계가 생성된 분할을 나타내기 위해 `split_names` 속성이 추가됩니다

In [ ]:
# grader-required-cell

# Get the latest TransformGraph artifact
statistics_artifact = store.get_artifacts_by_type('ExampleStatistics')[-1]

# Display the properties of the retrieved artifact
display_properties(store, statistics_artifact)

<a name='5-2'></a>
####5.2 - 아티팩트 추적

이 마지막 연습에서는 주어진 아티팩트의 부모 아티팩트를 반환하는 함수를 작성할 것입니다. 예를 들어, 특정 `TransformGraph` 인스턴스를 생성하는 데 사용된 아티팩트를 나열할 수 있어야 합니다.

<a name='ex-12'></a>
##### Exercise 12: 부모 아티팩트 가져오기

특정 아티팩트의 입력을 추적하기 위해 아래 코드를 완성하세요.

Tips:

* 여기 [get_events_by_artifact_ids()](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd/MetadataStore#get_events_by_artifact_ids) 와 [get_events_by_execution_ids()](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd/MetadataStore#get_executions_by_id) 를 유용하게 사용할 수 있습니다.

* MetadataStore 클래스의 일부 메서드들 (위에서 제공한 두 메서드 포함)은 iterable만 받아들이므로 단지 int만 있으면 리스트(또는 세트)로 변환하세요 (예: x 대신 [x]를 전달하세요).

In [ ]:
# grader-required-cell

def get_parent_artifacts(store, artifact):

    ### START CODE HERE ###

    # Get the artifact id of the input artifact
    artifact_id = artifact.id

    # Get events associated with the artifact id
    artifact_id_events = store.get_events_by_artifact_ids([artifact_id])

    # From the `artifact_id_events`, get the execution ids of OUTPUT events.
    # Cast to a set to remove duplicates if any.
    execution_id = set(
        event.execution_id
        for event in artifact_id_events
        if event.type == metadata_store_pb2.Event.OUTPUT
    )

    # Get the events associated with the execution_id
    execution_id_events = store.get_events_by_execution_ids(list(execution_id))

    # From execution_id_events, get the artifact ids of INPUT events.
    # Cast to a set to remove duplicates if any.
    parent_artifact_ids = set(
        event.artifact_id
        for event in execution_id_events
        if event.type == metadata_store_pb2.Event.INPUT
    )

    # Get the list of artifacts associated with the parent_artifact_ids
    parent_artifact_list = store.get_artifacts_by_id(list(parent_artifact_ids))

    ### END CODE HERE ###

    return parent_artifact_list

In [ ]:
# grader-required-cell

# Get an artifact instance from the metadata store
artifact_instance = store.get_artifacts_by_type('TransformGraph')[0]

# Retrieve the parent artifacts of the instance
parent_artifacts = get_parent_artifacts(store, artifact_instance)

# Display the results
display_artifacts(store, parent_artifacts, base_dir)

**Expected Output:**

*Note: The ID numbers may differ.*

| artifact id | type | uri |
| ----------- | ---- | --- |
| 1	| Examples | ./CsvExampleGen/examples/1 |
| 4	| Schema | ./ImportSchemaGen/schema/4 |

이 글에선 특징을 선택하고, 데이터 파이프라인을 수행하며, 메타데이터 저장소에서 정보를 검색하는 능력을 보여주었습니다. 프로덕션 수준의 머신 러닝 프로젝트에서 작업할 때 이 모든 것을 함께 사용하는 능력은 매우 중요합니다. 다음 주에는 더 많은 데이터 유형을 다루게 되며, ML 파이프라인에서 이러한 데이터를 어떻게 준비하는지 살펴볼 것입니다.

<details>
  <summary><font size="2" color="darkgreen"><b>Please click here if you want to experiment with any of the non-graded code.</b></font></summary>
    <p><i><b>Important Note: Please only do this when you've already passed the assignment to avoid problems with the autograder.</b></i>
    <ol>
        <li> On the notebook’s menu, click “View” > “Cell Toolbar” > “Edit Metadata”</li>
        <li> Hit the “Edit Metadata” button next to the code cell which you want to lock/unlock</li>
        <li> Set the attribute value for “editable” to:
            <ul>
                <li> “true” if you want to unlock it </li>
                <li> “false” if you want to lock it </li>
            </ul>
        </li>
        <li> On the notebook’s menu, click “View” > “Cell Toolbar” > “None” </li>
    </ol>
    <p> Here's a short demo of how to do the steps above:
        <br>
        <img src="https://drive.google.com/uc?export=view&id=14Xy_Mb17CZVgzVAgq7NCjMVBvSae3xO1" align="center">
</details>

In [56]:
from tfx.components import Trainer

...

trainer = Trainer(
    module_file="/content/penguin_utils_keras.py",
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000))

NameError: ignored